Begin by loading model and approximating Belitzky's minimal medium. 

In [1]:
## Preparing the GSM data ##
import pandas as pd
import cobra 
from IPython.display import display

# Load the model

model = cobra.io.read_sbml_model('../../models/subtilis_ iYO844.xml')

medium = model.medium
medium['EX_glu__L_e'] = 1
medium['EX_trp__L_e'] = 1
medium['EX_cit_e'] = 1
model.medium = medium


model.summary()



Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0008013,0,0.00%
cit_e,EX_cit_e,1,6,28.10%
fe3_e,EX_fe3_e,0.0008625,0,0.00%
glc__D_e,EX_glc__D_e,1.7,6,47.78%
glu__L_e,EX_glu__L_e,1,5,23.42%
h_e,EX_h_e,3.4,0,0.00%
k_e,EX_k_e,0.1766,0,0.00%
mg2_e,EX_mg2_e,0.02543,0,0.00%
nh4_e,EX_nh4_e,0.9755,0,0.00%
o2_e,EX_o2_e,9.832,0,0.00%


Lets get our gene protein reaction rules into a formate we can use to assess the mapping to our gene abundance data. 

In [2]:

rxn_gene_dict = {rxn.id: rxn.gene_name_reaction_rule for rxn in model.reactions}

# make it into a dataframe

rxn_gene_df = pd.DataFrame(rxn_gene_dict.items(), columns=['rxn_id', 'gene_names'])

#filter out reactions with no gene names

rxn_gene_df = rxn_gene_df[rxn_gene_df['gene_names'] != '']

df = rxn_gene_df

def parse_genes(genes):
    # Replace logical connectors with commas, and remove parentheses
    genes = genes.replace(' and ', ', ').replace(' or ', ', ').replace('(', '').replace(')', '')
    # Split on commas and strip extra spaces, then deduplicate
    genes = set([gene.strip() for gene in genes.split(',')])
    return ', '.join(sorted(genes))

# Apply the function to clean up gene names
df['gene_names'] = df['gene_names'].apply(parse_genes)

# If needed, aggregate by 'rxn_id' (if there are duplicate 'rxn_id' values)
df = df.groupby('rxn_id')['gene_names'].agg(lambda x: ', '.join(sorted(set(', '.join(x).split(', '))))).reset_index()

print(df.head(50))

       rxn_id                                gene_names
0     23CN2P1                                      yfkN
1     23CN2P2                                      yfkN
2     23CN2P3                                      yfkN
3     23CN2P4                                      yfkN
4      26DPAi                            spoVFA, spoVFB
5     2S6HCCi                                      menD
6     3AMBAt2                                      gabP
7     3NUCLE1                                      yfkN
8     3NUCLE2                                      yfkN
9     3NUCLE3                                      yfkN
10    3NUCLE4                                      yfkN
11    AAMYL_1                                      amyE
12     AB6PGH                                bglA, licH
13       ABTA                                      gabT
14      ABUTD                          aldX, aldY, dhaS
15    ABUTt2r                                      gabP
16    ACACT1r                                   

Now load in the abundance data and format so it is mappable to the previous dataframe. 


In [3]:
abundance = pd.read_excel('/Users/yahyafarooqi/Documents/AntiGEM/Data/intermediate/gene_set.xlsx')


display(abundance.head())



,GeneID,Abundance
0,ybdT,0.994016
1,ybxH,0.856044
2,ybdM,0.963332
3,ybdK,1.004663
4,ybxF,1.000806


In [4]:
# if geneID in abundance is in gene_names in df, add label "match" to geneID in abundance, if not add label "no_match"


def match_genes(genes, abundance):
    genes = set(genes.split(', '))
    return 'match' if genes & set(abundance['GeneID']) else 'no_match'

df['match'] = df['gene_names'].apply(match_genes, abundance=abundance)

#print(df.head(50))


In [5]:

# print the number of genes that match and do not match
print(df['match'].value_counts())




match
match       600
no_match    304
Name: count, dtype: int64


In [8]:
fluxes = model.optimize().fluxes

# filter by active reactions

print(len(fluxes))

short_fluxes = fluxes[fluxes.abs() > 1e-6]

print(len(short_fluxes))

print(short_fluxes.head())

#append these flux columns to the dataframe by reaction ID (first columb), so that flux value is added to the corresponding reaction ID

df_new = df.merge(short_fluxes.rename('flux'), left_on='rxn_id', right_index=True, how='left')  

#filter all rows with no flux value (NaN)

df_new = df_new[~df_new['flux'].isnull()]

#filter by match

df_new = df_new[df_new['match'] == 'match']

display(df_new.head(50))

print(len(df_new))

1250
337
EX_fe3_e      -0.000863
EX_glc__D_e   -1.700000
EX_glu__L_e   -1.000000
EX_ca2_e      -0.000801
EX_cit_e      -1.000000
Name: fluxes, dtype: float64


,rxn_id,gene_names,match,flux
5,2S6HCCi,menD,match,0.000067
16,ACACT1r,mmgA,match,0.006994
23,ACCOAC,"accA, accB, accC, accD, birA",match,0.254477
25,ACGK,argB,match,0.048256
27,ACHBS,"alsS, ilvB, ilvH",match,0.094915
28,ACKr,ackA,match,0.106779
29,ACLDC,alsD,match,0.004283
30,ACLS,"alsS, ilvB, ilvH",match,0.185334
31,ACOAD1,"acdA, fadE, mmgC, ydbM, yngJ",match,0.006994
32,ACOAD2,"acdA, mmgC",match,0.006994


215
